In [1]:
using Plots;
gadfly();

In [2]:
include("../fdtd/update.jl");
include("../fdtd/sources.jl");
include("../fdtd/boundaries.jl");
using update;
using sources;

In [3]:
#Global parameters
size = 200
endTime = 700
num_snaps = 350
snap_step = div(endTime, num_snaps)
#Grid
hy = zeros(size);
ez = zeros(size);
# output params
ez_snapshot = Array{Any}(num_snaps);
hy_snapshot = Array{Any}(num_snaps);

fieldsum = 1;
minsumtime = 1;
minfieldsnap = zeros(size);

In [4]:
#Time steps

for time in 1:endTime
    #Field update equations
    
    update.update_magnetic_field!(ez, hy)
    update.update_electric_field!(ez, hy)
    # Source at center
    sources.additive_gaussian_source!(ez, 50, time)

    #Boundaries
    boundaries.pmc_boundary!(hy, 200)
    boundaries.pec_boundary!(ez, 1)

    
    #Snapshots for animation
    if mod(time, snap_step) == 0
        ez_snapshot[div(time,snap_step)] = (time, copy(ez))
        hy_snapshot[div(time,snap_step)] = (time, copy(hy).*globals.imp0)        
    end
    
end

In [5]:
anim = Animation()

for i = 1:num_snaps
    p = plot(1:size, ez_snapshot[i][2], lab="Ez")
    plot!(1:size, hy_snapshot[i][2], lab="Hy*imp0")
    
    time = ez_snapshot[i][1]
    plot!(ann=[(150, 1.5, "time =$time")])
    plot!(ann=[(0, 1.1, "PEC")])
    plot!(ann=[(180, 1.1, "PMC")])    
    plot!(xlims=(1, 200), ylims=(-2, 2))
    frame(anim, p)
end
gif(anim, "./02_additive.gif", fps=15)

[Plots.jl] Initializing backend: gadfly


INFO: Saved animation to /media/storage/Documents/Github/1d-fdtd/examples/02_additive.gif


Plots.AnimatedGif("/media/storage/Documents/Github/1d-fdtd/examples/02_additive.gif")